In [ ]:
# Importing libraries

import language_check
import numpy as np
import os
import pandas as pd
import sys, re, os, nltk
import requests
import warnings
from nltk import word_tokenize
from nltk.corpus import words, wordnet
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tree import Tree
from pandas.core.common import SettingWithCopyWarning
from stanfordcorenlp import StanfordCoreNLP
from textstat.textstat import textstat

nlp = StanfordCoreNLP(r'C:\Users\hp word\anaconda3\Lib\site-packages\stanfordcorenlp')
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Common functions

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
output_list = []
wordset = set(words.words())
lmtzr = WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
tool = language_check.LanguageTool('en-US')

def restrict2TwoDecimals(D):
    return round(D, 2)

def handleDivByZero(n_1, n_2):
    if n_2 == 0.0:
        return 0.0;
    elif n_1/n_2 > 10000:
        return 0.0;
    else:
        return restrict2TwoDecimals(n_1/n_2)

# Reading output of previous txt to csv converter

In [ ]:
testing_data = pd.read_csv('test_data.csv' ,encoding='ANSI')
test_text = testing_data

# Readability Features

In [ ]:
# Using textstat library implementation of readability features
for index, row in test_text.iterrows():
    essay = row['text'] 
    fre = textstat.flesch_reading_ease(essay)
    fkg = textstat.flesch_kincaid_grade(essay)
    cli = textstat.coleman_liau_index(essay)
    ari = textstat.automated_readability_index(essay)
    dcrs = textstat.dale_chall_readability_score(essay)
    dw = textstat.difficult_words(essay)
    lwf = textstat.linsear_write_formula(essay)
    gf = textstat.gunning_fog(essay)
    test_text.at[index, 'fre'] = fre
    test_text.at[index, 'fkg'] = fkg
    test_text.at[index, 'cli'] = cli
    test_text.at[index, 'ari'] = ari
    test_text.at[index, 'dcrs'] = dcrs
    test_text.at[index, 'dw'] = dw
    test_text.at[index, 'lwf'] = lwf
    test_text.at[index, 'gf'] = gf
test_text

# Word Level Features

In [ ]:
# Method to get Measure of Textual Lexical Diversity
def getMTLD(tokens):
    types = []
    factors=0
    ttrThreshold = 0.72
    startIndex = 0
    ttr=1
    # go over the text and get ttr to get the number of factors
    for i in range(len(tokens)):
        currentToken = tokens[i]
        #each time a new type is found, compute type token ratio
        if currentToken.lower() not in types:
            types.append(currentToken.lower())
        ttr = len(types)/(i+1-startIndex)
        if ttr < ttrThreshold:
       #cut text (those portions are called factor) and reset list of types
            startIndex = i+1
            types.clear()
            #keep count of factors
            factors +=1        
        #if it is the last word and the ttr threshold is not reached, calculate the rest factor
        elif (ttr >ttrThreshold) and (i ==len(tokens)-1):
            factors += (1-ttr)/(1-0.72)
    #repeat until all tokens are finished.
    #form MTLD score:  #tokens /#factors
    mtld1 = len(tokens) / factors
    
    #repeat same starting at the end of the text
    factors = 0
    startIndex= len(tokens)-1
    ttr=1
    types.clear()
    for i in reversed(range(len(tokens))):
        currentToken = tokens[i]
        #each time a new type is found, compute type token ratio
        if currentToken.lower() not in types:
            types.append(currentToken.lower())
        ttr = len(types)/(startIndex-i+1)

        #when ttr reaches threshold
        if ttr < ttrThreshold:
            #cut text (those portions are called factor) and reset list of types 
            startIndex = i-1
            types.clear()
            factors +=1
        #if it is the last word and the ttr threshold is not reached. calculate the rest factor
        elif (ttr >ttrThreshold) and (i ==0):
            factors += (1-ttr)/(1-0.72)
            
    #repeat until not tokens left

    mtld2 = len(tokens)/ factors
    res = (mtld1+mtld2)/2
    resD = float(res)
    #take the mean of both forward and backward score
    if not np.isinf(resD):
        return resD
    else:
        return 0.0

In [ ]:
for index, row in test_text.iterrows():
    essay_low = row['text'].lower()
    essay_low_np = essay_low.replace('[^\w\s]','')
    tokens_low_np = tokenizer.tokenize(essay_low_np)
    types = set(tokens_low_np)
    num_types = len(types)
    num_tokens = len(tokens_low_np)
    
    test_text.at[index, 'Word_numWords'] = num_tokens
    test_text.at[index, 'Word_TTR'] = handleDivByZero(num_types, num_tokens)
    test_text.at[index, 'Word_CTTR'] = handleDivByZero(num_types, np.sqrt(2.0*num_tokens))
    test_text.at[index, 'Word_RTTR'] = handleDivByZero(num_types,np.sqrt(num_tokens))
    test_text.at[index, 'Word_BilogTTR'] = handleDivByZero(np.log(num_types),np.log(num_tokens))
    test_text.at[index, 'Word_UberIndex'] = handleDivByZero(np.log(num_tokens)**2,np.log(num_tokens/num_types))
    test_text.at[index, 'Word_MTLD'] = restrict2TwoDecimals(getMTLD(tokens_low_np))
    
test_text

# Error features

In [ ]:
# Used language_check library implementation for error detection
for index, row in test_text.iterrows():
    essay = row['text']
    sentences = sent_tokenize(essay)
    num_sents = len(sentences)
    spelling_mistake = 0
    duplicate_mistake = 0
    other_mistake = 0
    for sentence in sentences:
        matches = tool.check(sentence)
        for match in matches:
            if match.locqualityissuetype == 'misspelling':
                spelling_mistake +=1
            elif match.locqualityissuetype == 'duplication':
                duplicate_mistake +=1
            else:
                other_mistake +=1
                
    all_mistakes = spelling_mistake + duplicate_mistake + other_mistake
    
    test_text.at[index, 'SpellingErrorsPerSen'] = restrict2TwoDecimals(spelling_mistake/num_sents)
    test_text.at[index, 'DuplicateErrorsPerSen'] = restrict2TwoDecimals(duplicate_mistake/num_sents)
    test_text.at[index, 'OtherErrorsPerSen'] = restrict2TwoDecimals(other_mistake/num_sents)
    test_text.at[index, 'AllErrorsPerSen'] = restrict2TwoDecimals(all_mistakes/num_sents)
    test_text.at[index, 'SpellingErrorsWrtAllErrors'] = restrict2TwoDecimals(spelling_mistake/all_mistakes)
test_text

# Parts Of Speech (POS) Features

In [ ]:
# Used NLTK library implementation of POS features
# Run 'nltk.help.upenn_tagset()' to get list of The Penn Treebank's POS tags

for index, row in test_text.iterrows():
    essay = row['text']
    tokens = word_tokenize(essay)
    tagged = nltk.pos_tag(tokens)
    TotalWords = 0;
    numAdj = 0;
    numNouns = 0;
    numVerbs = 0;
    numPronouns = 0;
    numConjunct = 0;
    numProperNouns = 0;	 
    numPrepositions = 0;
    numAdverbs = 0;
    numLexicals = 0;
    numModals = 0;
    numInterjections = 0;
    perpronouns = 0;
    whperpronouns = 0;
    numauxverbs = 0;
    numFunctionWords = 0;
    numDeterminers = 0;
    numVB = 0;
    numVBD = 0;
    numVBG = 0;
    numVBN = 0;
    numVBP = 0;
    numVBZ = 0;
    uniqueVerbs = []
    for word, tag in tagged:
        if tag =="PRP" or tag =="PRP$" or tag=="WP" or tag=="WP$":
            numPronouns +=1
            if tag == "PRP":
                perpronouns +=1
            if tag == "WP":
                whperpronouns +=1
            numFunctionWords +=1
            TotalWords +=1
            
        if tag == "VB" or tag == "VBD" or tag == "VBG" or tag == "VBN" or tag == "VBP" or tag == "VBZ":
            numVerbs+=1
            TotalWords+=1
            if word not in uniqueVerbs:
                uniqueVerbs.append(word)
            if tag == "VB":
                numVB+=1
            elif tag == "VBD":
                numVBD+=1
            elif tag == "VBG":
                numVBG+=1
            elif tag == "VBN":
                numVBN+=1
            if tag == "VBP":
                numVBP+=1
            if tag == "VBZ":
                numVBZ+=1
        if tag == "JJ" or tag == "JJR" or tag == "JJS":
            numAdj+=1
            TotalWords+=1
        if tag == "RB" or tag == "RBR" or tag == "RBS" or tag == "RP":
            numAdverbs+=1
            numFunctionWords+=1
            TotalWords+=1
        if tag == "IN":
            numPrepositions+=1
            numFunctionWords+=1
            TotalWords+=1
        if tag == "UH":
            numInterjections+=1
            numFunctionWords+=1
            TotalWords+=1
        if tag == "CC":
            numConjunct+=1
            numFunctionWords+=1
            TotalWords+=1
        if tag == "NN" or tag == "NNS":
            numNouns+=1
            TotalWords+=1
        if tag == "NNP" or tag == "NNPS":
            numProperNouns+=1
            TotalWords+=1
        if tag == "MD":
            numModals+=1
            numauxverbs+=1
            numFunctionWords+=1
            TotalWords+=1
        if tag == "DT":
            numFunctionWords+=1
            numDeterminers+=1
            TotalWords+=1
        #End of all words in a sentence.
    #End of all sentences
    numLexicals = numAdj+numNouns+numVerbs+numAdverbs+numProperNouns
    numVerbsOnly = numVerbs-numauxverbs

    test_text.at[index, 'POS_numNouns'] = restrict2TwoDecimals(numNouns+numProperNouns/TotalWords)
    test_text.at[index, 'POS_numProperNouns'] = restrict2TwoDecimals(numProperNouns/TotalWords)
    test_text.at[index, 'POS_numPronouns'] = restrict2TwoDecimals(numPronouns/TotalWords)
    test_text.at[index, 'POS_numConjunct'] = restrict2TwoDecimals(numConjunct/TotalWords)
    test_text.at[index, 'POS_numAdjectives'] = restrict2TwoDecimals(numAdj/TotalWords)
    test_text.at[index, 'POS_numVerbs'] = restrict2TwoDecimals(numVerbs/TotalWords)
    test_text.at[index, 'POS_numAdverbs'] = restrict2TwoDecimals(numAdverbs/TotalWords)
    test_text.at[index, 'POS_numModals'] = restrict2TwoDecimals(numModals/TotalWords)
    test_text.at[index, 'POS_numPrepositions'] = restrict2TwoDecimals(numPrepositions/TotalWords)
    test_text.at[index, 'POS_numInterjections'] = restrict2TwoDecimals(numInterjections/TotalWords)
    test_text.at[index, 'POS_numPerPronouns'] = restrict2TwoDecimals(perpronouns/TotalWords)
    test_text.at[index, 'POS_numWhPronouns'] = restrict2TwoDecimals(whperpronouns/TotalWords)
    test_text.at[index, 'POS_numLexicals'] = restrict2TwoDecimals((numLexicals)/TotalWords)
    test_text.at[index, 'POS_numFunctionWords'] = restrict2TwoDecimals((numFunctionWords)/TotalWords)
    test_text.at[index, 'POS_numDeterminers'] = restrict2TwoDecimals((numDeterminers)/TotalWords)
    test_text.at[index, 'POS_numVerbsVB'] = restrict2TwoDecimals((numVB)/TotalWords)
    test_text.at[index, 'POS_numVerbsVBD'] = restrict2TwoDecimals((numVBD)/TotalWords)
    test_text.at[index, 'POS_numVerbsVBG'] = restrict2TwoDecimals((numVBG)/TotalWords)
    test_text.at[index, 'POS_numVerbsVBN'] = restrict2TwoDecimals((numVBN)/TotalWords)
    test_text.at[index, 'POS_numVerbsVBP'] = restrict2TwoDecimals((numVBP)/TotalWords)
    test_text.at[index, 'POS_numVerbsVBZ'] = restrict2TwoDecimals((numVBZ)/TotalWords)
    test_text.at[index, 'POS_advVar'] = restrict2TwoDecimals(numAdverbs/numLexicals)
    test_text.at[index, 'POS_adjVar'] = restrict2TwoDecimals(numAdj/numLexicals)
    test_text.at[index, 'POS_modVar'] = restrict2TwoDecimals((numAdj+numAdverbs)/numLexicals)
    test_text.at[index, 'POS_nounVar'] = restrict2TwoDecimals((numNouns+numProperNouns)/numLexicals)
    test_text.at[index, 'POS_verbVar1'] = restrict2TwoDecimals((numVerbsOnly)/len(uniqueVerbs))
    test_text.at[index, 'POS_verbVar2'] = restrict2TwoDecimals((numVerbsOnly)/numLexicals)
    test_text.at[index, 'POS_squaredVerbVar1'] = restrict2TwoDecimals((numVerbsOnly*numVerbsOnly)/len(uniqueVerbs))
    test_text.at[index, 'POS_correctedVV1'] = restrict2TwoDecimals((numVerbsOnly)/np.sqrt(2.0*len(uniqueVerbs)))
    
test_text

# Syntactic Features

In [ ]:
# This code generates parse tree for the given text and make syntactic calculations.
for index, row in test_text.iterrows():
    essay = row['text']
    sentences = sent_tokenize(essay)
    numSBAR = 0
    avgParseTreeHeight = 0
    numNP = 0
    numVP = 0
    numPP = 0
    numSubtrees = 0
    numWhPhrases = 0
    numConjPhrases = 0
    reducedRelClauses = 0
    numWords = 0
    numClauses = 0
    numTunits = 0
    numComplexNominals = 0
    numDependentClauses = 0
    numCoordinateClauses = 0
    numComplexTunits = 0
    AvgNPSize = 0
    AvgVPSize = 0
    AvgPPSize = 0
    
    for sentence in sentences:
        try:
            cn_tree = nlp.parse(sentence)
            tree = Tree.fromstring(cn_tree)
            avgParseTreeHeight += tree.height()
            numWords += len(tree.leaves())
            for st in tree.subtrees():
                numSubtrees +=1
                if st.label() == "NP":
                    numNP += 1
                    AvgNPSize += len(st)

                if st.label() == "VP":
                    numVP += 1
                    AvgVPSize += len(st)

                if st.label() == "PP":
                    numPP += 1
                    AvgPPSize += len(st)

                if st.label() == "WHNP" or st.label() == "WHPP" or st.label() == "WHADVP" or st.label() == "WHADJP":
                    numWhPhrases += 1

                if st.label() == "RRC":
                    reducedRelClauses += 1

                if st.label() == "CONJP":
                    numConjPhrases += 1
        except:
            continue
            
    numSentences = len(sentences)
    
    test_text.at[index, 'SYN_numSentences'] = numSentences
    test_text.at[index, 'SYN_avgSentenceLength'] = restrict2TwoDecimals(numWords/numSentences)
    test_text.at[index, 'SYN_avgParseTreeHeightPerSen'] = restrict2TwoDecimals(avgParseTreeHeight/numSentences)
    test_text.at[index, 'SYN_numSubtreesPerSen'] = restrict2TwoDecimals(numSubtrees/numSentences)
    test_text.at[index, 'SYN_numNPsPerSen'] = restrict2TwoDecimals(numNP/numSentences)
    test_text.at[index, 'SYN_numVPsPerSen'] = restrict2TwoDecimals(numVP/numSentences)
    test_text.at[index, 'SYN_numPPsPerSen'] = restrict2TwoDecimals(numPP/numSentences)
    test_text.at[index, 'SYN_numNPSize'] = handleDivByZero(AvgNPSize,numNP)
    test_text.at[index, 'SYN_numVPSize'] = handleDivByZero(AvgVPSize,numVP)
    test_text.at[index, 'SYN_numPPSize'] = handleDivByZero(AvgPPSize,numPP)
    test_text.at[index, 'SYN_numWHPsPerSen'] = restrict2TwoDecimals(numWhPhrases/numSentences)
    test_text.at[index, 'SYN_numRRCsPerSen'] = restrict2TwoDecimals(reducedRelClauses/numSentences)
    test_text.at[index, 'SYN_numConjPPerSen'] = restrict2TwoDecimals(numConjPhrases/numSentences)
    
test_text   

# Discourse Features

### Referring Expressions

In [ ]:
# This code generates discourse features related to referring expressions
for index, row in test_text.iterrows():
    essay = row['text']
    sentences = sent_tokenize(essay)
    tokens = word_tokenize(essay)
    tagged = nltk.pos_tag(tokens)
    numWords = 0.0
    numSentences = len(sentences)
    numPronouns = 0.0
    numPersonalPronouns = 0.0
    numPossessivePronouns = 0.0
    numDefiniteArticles = 0.0
    numProperNouns = 0.0
    numNouns = 0.0
    for word, tag in tagged:
        if word[0].isalpha():
            numWords +=1
            if tag == 'DT' and word.lower() == 'the':
                numDefiniteArticles +=1
            elif tag == 'PRP':
                numPersonalPronouns +=1
            elif tag == 'PRP$':
                numPossessivePronouns +=1
            elif tag.startswith('NN'):
                numNouns +=1
                if tag.startswith('NNP'):
                    numProperNouns +=1
                    
    numPronouns = numPersonalPronouns + numPossessivePronouns
    
    test_text.at[index, 'DISC_RefExprPronounsPerNoun'] = restrict2TwoDecimals(numPronouns/numNouns)
    test_text.at[index, 'DISC_RefExprPronounsPerSen'] = restrict2TwoDecimals(numPronouns/numSentences)
    test_text.at[index, 'DISC_RefExprPronounsPerWord'] = restrict2TwoDecimals(numPronouns/numWords)
    test_text.at[index, 'DISC_RefExprPerPronounsPerSen'] = restrict2TwoDecimals(numPersonalPronouns/numSentences)
    test_text.at[index, 'DISC_RefExprPerProPerWord'] = restrict2TwoDecimals(numPersonalPronouns/numWords)
    test_text.at[index, 'DISC_RefExprPossProPerSen'] = restrict2TwoDecimals(numPossessivePronouns/numSentences)
    test_text.at[index, 'DISC_RefExprPossProPerWord'] = restrict2TwoDecimals(numPossessivePronouns/numWords)
    test_text.at[index, 'DISC_RefExprDefArtPerSen'] = restrict2TwoDecimals(numDefiniteArticles/numSentences)
    test_text.at[index, 'DISC_RefExprDefArtPerWord'] = restrict2TwoDecimals(numDefiniteArticles/numWords)
    test_text.at[index, 'DISC_RefExprProperNounsPerNoun'] = restrict2TwoDecimals(numProperNouns/numNouns)

test_text

### Content Overlap features

In [ ]:
def getGeneralTag(specificTag):
    generaltag = "NOTAG"
    if specificTag.startswith("VB"): 
        generaltag = "VERB"

    elif specificTag.startswith("JJ"):
        generaltag = "ADJECTIVE"

    elif specificTag.startswith("RB") or specificTag == "WRB" :
        generaltag = "ADVERB"
        
    elif specificTag.startswith("PRP") or specificTag.startswith("WP"):
        generaltag = "PRONOUN"

    elif specificTag.startswith("NN"):
        generaltag = "NOUN"

    return generaltag

def lemma_function(word, tag):
    lemmatizer = WordNetLemmatizer()
    return lemmatizer.lemmatize(word)
    
def returnFormattedSentence(sentences):
    formated_sent = []
    for sentence in sentences:
        tokens = word_tokenize(sentence)
        tagged = nltk.pos_tag(tokens)
        single_sent = []
        for word, tag in tagged:
            word_prop = []
            word_prop.append(word.lower())
            word_prop.append(lemma_function(word, tag))
            word_prop.append(tag)
            word_prop.append(getGeneralTag(tag))
            single_sent.append(word_prop)
        formated_sent.append(single_sent)
    return formated_sent

def isThereNounOverlap(sent_1, sent_2):
    for word in sent_1:
        if (word[3]=="NOUN") and (word in sent_2):
            return True
    return False

def isThereArgumentOverlap(sent_1, sent_2):
    if isThereNounOverlap(sent_1, sent_2):
        return True
    else:
        for word in sent_1:
            if (word[3]=="PRONOUN") and (word in sent_2):
                return True
            else:
                if word[3]=="NOUN" or word[3]=="PRONOUN":
                    word_lemma_1 = word[1]
                    word_pos_1 = word[3]
                    for word2 in sent_2:
                        word_lemma_2 = word2[1]
                        word_pos_2 = word2[3]
                        if (word_lemma_1 == word_lemma_2) and (word_pos_1 == word_pos_2) and (not word_pos_1 == "NOTAG"):
                            return True
    return False

def isThereStemOverlap(sent_1, sent_2):
    if isThereNounOverlap(sent_1, sent_2) or isThereArgumentOverlap(sent_1, sent_2):
        return True
    else:
        for word in sent_1:
            if not word[3] == "NOTAG":
                word_lemma_1 = word[1]
                word_pos_1 = word[3]
                for word2 in sent_2:
                    word_lemma_2 = word2[1]
                    word_pos_2 = word2[3]
                    if (word_lemma_1 == word_lemma_2) and (word_pos_1=="NOUN" or word_pos_2=="NOUN" or word_pos_1 == "PRONOUN"):
                        return True
    return False

def contentWordOverlap(sent_1, sent_2):
    overlapsCount = 0
    for word in sent_1:
        word_lemma_1 = word[1]
        word_pos_1 = word[3]
        if (not word_pos_1 == "NOTAG") and (not word_pos_1 == "PRONOUN"):
            for word2 in sent_2:
                if word_lemma_1 == word2[1]:
                    overlapsCount +=1
    return overlapsCount

In [ ]:
for index, row in test_text.iterrows():
    essay = row['text']
    sentences = sent_tokenize(essay)
    formatted_sentences = returnFormattedSentence(sentences)
    localNounOverlapCount = 0
    localArgumentOverlapCount = 0
    localStemOverlapCount = 0
    localContentWordOverlap = 0

    globalNounOverlapCount = 0
    globalArgumentOverlapCount = 0
    globalStemOverlapCount = 0
    globalContentWordOverlap = 0
    
    totalSentencesSize = len(sentences)
    for i in range(0,totalSentencesSize):
        for j in range(i+1,totalSentencesSize):
            sent_1, sent_2 = formatted_sentences[i], formatted_sentences[j]
            if isThereNounOverlap(sent_1, sent_2):
                if (j-i) == 1:
                    localNounOverlapCount +=1
                    localArgumentOverlapCount +=1
                    localStemOverlapCount +=1
                globalNounOverlapCount +=1
                globalArgumentOverlapCount +=1
                globalStemOverlapCount +=1
            elif isThereArgumentOverlap(sent_1, sent_2):
                if (j-i) ==1:
                    localArgumentOverlapCount +=1
                    localStemOverlapCount +=1
                globalArgumentOverlapCount +=1
                globalStemOverlapCount +=1
            elif isThereStemOverlap(sent_1, sent_2):
                if (j-i) ==1:
                    localStemOverlapCount +=1
                globalStemOverlapCount +=1
            tempContentOverlap = contentWordOverlap(sent_1, sent_2)
            globalContentWordOverlap += tempContentOverlap
            if (j-i) ==1:
                localContentWordOverlap += tempContentOverlap
    test_text.at[index, 'total_sentences'] = totalSentencesSize
    test_text.at[index, 'DISC_localNounOverlapCount'] = restrict2TwoDecimals(localNounOverlapCount/totalSentencesSize)
    test_text.at[index, 'DISC_localArgumentOverlapCount'] = restrict2TwoDecimals(localArgumentOverlapCount/totalSentencesSize)
    test_text.at[index, 'DISC_localStemOverlapCount'] = restrict2TwoDecimals(localStemOverlapCount/totalSentencesSize)
    test_text.at[index, 'DISC_localContentWordOverlapCount'] = restrict2TwoDecimals(localContentWordOverlap/totalSentencesSize)
    test_text.at[index, 'DISC_globalNounOverlapCount'] = restrict2TwoDecimals(globalNounOverlapCount/totalSentencesSize)
    test_text.at[index, 'DISC_globalArgumentOverlapCount'] = restrict2TwoDecimals(globalArgumentOverlapCount/totalSentencesSize)
    test_text.at[index, 'DISC_globalStemOverlapCount'] = restrict2TwoDecimals(globalStemOverlapCount/totalSentencesSize)
    test_text.at[index, 'DISC_globalContentWordOverlapCount'] = restrict2TwoDecimals(globalContentWordOverlap/totalSentencesSize)

test_text

In [ ]:
# path to Discourse output text marked with connective tags.
path = 'C:\\Users\\hp word\\Documents\\University of Bath Labs\\SEM 2\\Research project dissertation\\Codes_Libraries\\My_code\\Feature_engineering\\DiscourseFeaturesData\\TestDiscOutputTxt'

# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        with open (str(os.path.join(r, file)), "r") as myfile:
            data=myfile.readlines()
            
            numNonDiscConnectives = len([k for k in data if '#0' in k])
            numCompConnectives = len([k for k in data if '#Comparison' in k])
            numExpConnectives = len([k for k in data if '#Expansion' in k])
            numContConnectives = len([k for k in data if '#Contingency' in k])
            numTempConnectives = len([k for k in data if '#Temporal' in k])
            
            numDiscConnectives = numCompConnectives + numExpConnectives + numContConnectives + numTempConnectives;
            numConnectives = numDiscConnectives + numNonDiscConnectives;
        
            numSentences = test_text.loc[test_text['doc_id'] == str(file)]['total_sentences'].values[0]
            test_text.loc[test_text['doc_id'] == str(file),'DISCPlus_00_numConnectivesPerSen'] = restrict2TwoDecimals(numConnectives/numSentences)
            test_text.loc[test_text['doc_id'] == str(file),'DISCPlus_01_numDiscConnectivesPerSen'] = restrict2TwoDecimals(numDiscConnectives/numSentences)
            test_text.loc[test_text['doc_id'] == str(file),'DISCPlus_02_numNonDiscConnectivesPerSen'] = restrict2TwoDecimals(numNonDiscConnectives/numSentences)
            test_text.loc[test_text['doc_id'] == str(file),'DISCPlus_03_numCompConnectivesPerSen'] = restrict2TwoDecimals(numCompConnectives/numSentences)
            test_text.loc[test_text['doc_id'] == str(file),'DISCPlus_04_numExpConnectivesPerSen'] = restrict2TwoDecimals(numExpConnectives/numSentences)
            test_text.loc[test_text['doc_id'] == str(file),'DISCPlus_05_numContConnectives'] = restrict2TwoDecimals(numContConnectives/numSentences)
            myfile.close()
test_text

# Creating a csv file

In [ ]:
# Removing a duplicate feature
test_text = test_text.drop("total_sentences", axis = 1)
test_text.to_csv(r'test_features - Copy.csv', index = False)